In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import pandas
import spektral
import random
from dataclasses import dataclass
from spektral.utils import tic, toc

In [2]:
!pip show tensorflow

Name: tensorflow
Version: 2.6.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\conno\appdata\roaming\python\python38\site-packages
Requires: wheel, numpy, tensorflow-estimator, h5py, six, absl-py, astunparse, clang, wrapt, google-pasta, flatbuffers, termcolor, opt-einsum, keras, tensorboard, gast, grpcio, typing-extensions, protobuf, keras-preprocessing
Required-by: spektral


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2679603500104633918
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4160159744
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13292143627832581356
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
random.seed(69420)

In [5]:
#Spatio Temporal Graph Dataset
@dataclass()
class STG_Dataset:
    adjs: np.ndarray
    adjs_timestep: np.ndarray
    feats: np.ndarray
    feats_timestep: np.ndarray
    labels: np.ndarray
    labels_timestep: np.ndarray

    n_nodes: int
    n_timestamps: int
    n_class: int
    n_feat: int

    idx_train: np.ndarray
    idx_val: np.ndarray
    idx_test: np.ndarray

In [6]:
def read_data(id: str): #DBLP3, DBLP5, Brain, Reddit, DBLPE
    dataset_dict=dict()
    dataset_dict["DBLP3"]="Datasets/DBLP3.npz"
    dataset_dict["DBLP5"]="Datasets/DBLP5.npz"
    dataset_dict["Brain"]="Datasets/Brain.npz"
    dataset_dict["Reddit"]="Datasets/reddit.npz"
    dataset_dict["DBLPE"]="Datasets/DBLPE.npz"

    dataset = np.load(dataset_dict[id])
    adjs = dataset["adjs"] #(time, node, node)

    #Remove nodes with no connections at any timestep
    temporal_sum = tf.math.reduce_sum(adjs, axis=0, keepdims=False, name=None)
    row_sum = tf.math.reduce_sum(temporal_sum, axis=0, keepdims=False, name=None)
    non_zero_indices = np.flatnonzero(row_sum)
    adjs = adjs[:,non_zero_indices,:]
    adjs = adjs[:,:,non_zero_indices]

    #DBLPE is a dynamic featureless graph
    if id=="DBLPE":
        labels = dataset["labels"] #(nodes, time, class)

        # labels = np.argmax(labels,axis=2)
        labels=labels[non_zero_indices]
        feats=np.zeros([adjs.shape[1], adjs.shape[0], adjs.shape[2]])

        for i in range(feats.shape[1]):
            feats[:,i,:]=np.eye(feats.shape[0])
      
    #All others are static feature-full graphs
    else:
        labels = dataset["labels"] #(nodes, class)
        feats = dataset["attmats"] #(node, time, feat)

        # labels = np.argmax(labels, axis=1)
        labels = labels[non_zero_indices]
        feats = feats[non_zero_indices]

    #Other important variables
    n_nodes = adjs.shape[1]
    n_timesteps = adjs.shape[0]
    n_class = int(labels.shape[1])
    n_feat = feats.shape[2]

    #Train Val Test split
    nodes_id = list(range(n_nodes))
    random.shuffle(nodes_id)
    idx_train = nodes_id[:(7*n_nodes)//10]
    idx_train = [True if i in idx_train else False for i in list(range(n_nodes))]
    idx_val = nodes_id[(7*n_nodes)//10: (9*n_nodes)//10]
    idx_val = [True if i in idx_val else False for i in list(range(n_nodes))]
    idx_test = nodes_id[(9*n_nodes)//10: n_nodes]
    idx_test = [True if i in idx_test else False for i in list(range(n_nodes))]

    return STG_Dataset(tf.convert_to_tensor(adjs,dtype=tf.float32),
                        tf.convert_to_tensor(adjs,dtype=tf.float32),
                        tf.convert_to_tensor(feats,dtype=tf.float32), 
                        tf.convert_to_tensor(feats,dtype=tf.float32), 
                        tf.convert_to_tensor(labels,dtype=tf.float32), 
                        tf.convert_to_tensor(labels,dtype=tf.float32), 
                        n_nodes, n_timesteps, n_class, n_feat, 
                        np.array(idx_train),
                        np.array(idx_val),
                        np.array(idx_test))
    

In [7]:
class Gat(tf.keras.Model):
    def __init__(self, nhid, nclass, dropout):
        super(Gat, self).__init__()

        self.dropout = tf.keras.layers.Dropout(dropout)
        self.gat1 = spektral.layers.GATConv(channels=nhid,attn_heads=1,concat_heads=True, activation='relu')
        self.gat2 = spektral.layers.GATConv(channels=nclass,attn_heads=1,concat_heads=True, activation='softmax')

    def call(self, inputs, training=None):
        feats, adj = inputs
        x_1 = self.gat1([feats, adj])
        dropout = self.dropout(x_1, training=training)
        return self.gat2([dropout, adj])

    # def model(self, n_feat, n_nodes):
    #     x_in = tf.keras.layers.Input(shape=(n_feat,))
    #     a_in = tf.keras.layers.Input(shape=(None,), sparse=True)
    #     return tf.keras.Model(inputs=[x_in, a_in], outputs=self.call([x_in, a_in]))

In [8]:
# Training step
@tf.function
def train(feats, adjs, labels, idx_train, idx_val, model, loss_fn, optimizer, acc):
    #training
    with tf.GradientTape() as tape:
        predictions = model([feats, adjs], training=True)
        loss_train = loss_fn(labels[idx_train], predictions[idx_train])
    gradients = tape.gradient(loss_train, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    #evaluating
    predictions = model([feats, adjs], training=False)
    loss_val = loss_fn(labels[idx_val], predictions[idx_val])

    acc.update_state(labels[idx_val], predictions[idx_val])


@tf.function
def test(feats, adjs, labels, idx_test, model, loss_fn, optimizer, acc, auc, f1):
    predictions = model([feats, adjs], training=False)
    loss_test = loss_fn(labels[idx_test], predictions[idx_test])

        
    acc.update_state(labels[idx_test], predictions[idx_test])
    auc.update_state(labels[idx_test], predictions[idx_test])
    f1.update_state(labels[idx_test], predictions[idx_test])
    return loss_test

In [9]:
def timestep_train_test(epochs, model, data, loss_fn, optimizer, val_acc, acc, auc, f1):
    best_val=0
    tic()
    for epoch in range(1, epochs + 1):
        train(data.feats_timestep, data.adjs_timestep, data.labels_timestep, data.idx_train, data.idx_val, model, loss_fn, optimizer, val_acc)
        if val_acc.result() > best_val:
            best_val = val_acc.result()
        val_acc.reset_state()
    print(f"Best Val Acc: {best_val}")

    loss_test = test(data.feats_timestep, data.adjs_timestep, data.labels_timestep, data.idx_test, model, loss_fn, optimizer, acc, auc, f1)
    print(f"Test Loss: {loss_test}, Test Acc: {acc.result()}, Test F1 score: {f1.result()}, Auc Test: {auc.result()}")
    toc(f"{model.name} ({epochs} epochs)")

In [10]:
def overall_train_test(data_id, model_id):
    #Constant parameters
    epochs = 500

    data = read_data(data_id)
    model = model_id(data.n_class, data.n_class, 0.5)
    model.build([(data.n_nodes, data.n_feat), (data.n_nodes, data.n_nodes)])
    model.summary()
    optimizer = tfa.optimizers.AdamW(learning_rate=25e-4, weight_decay=5e-4)
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    val_acc = tf.keras.metrics.CategoricalAccuracy()
    acc = tf.keras.metrics.CategoricalAccuracy()
    auc = tf.keras.metrics.AUC(num_thresholds=data.adjs.shape[0], multi_label=True)
    f1 = tfa.metrics.FBetaScore(data.labels.shape[1], average="weighted")

    for timestep in range(data.n_timestamps + 1):
        data.adjs_timestep = tf.identity(data.adjs[:timestep,:,:])
        data.feats_timestep = tf.identity(data.feats)
        data.labels_timestep = tf.identity(data.labels)
        if (data_id == "DBLPE"):
            data.labels_timestep = data.labels_timestep[:,timestep - 1]
        
        #If the model ignores temporal data, accumulate adj matrices
        if (model_id.__name__ in ["Gat"]):
            data.adjs_timestep = tf.math.reduce_sum(data.adjs_timestep, axis=0, keepdims=False, name=None)
            data.feats_timestep = data.feats_timestep[:, -1, :]

            #normalize the adj matrix
            data.adjs_timestep += tf.eye(data.adjs_timestep.shape[0])
            d = tf.reduce_sum(data.adjs_timestep, axis=1)
            normalizing_matrix = np.zeros((data.adjs_timestep.shape[0], data.adjs_timestep.shape[0]))
            normalizing_matrix[range(len(normalizing_matrix)), range(len(normalizing_matrix))] = d**(-0.5)
            normalizing_matrix = tf.convert_to_tensor(normalizing_matrix, dtype=tf.float32)
            data.adjs_timestep = tf.matmul(normalizing_matrix,data.adjs_timestep)
            data.adjs_timestep=tf.matmul(tf.matmul(normalizing_matrix,data.adjs_timestep), normalizing_matrix)

        timestep_train_test(epochs, model, data, loss_fn, optimizer, val_acc, acc, auc, f1)
            


In [11]:
overall_train_test("DBLP3", Gat)

Model: "gat"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            multiple                  0         
_________________________________________________________________
gat_conv (GATConv)           multiple                  309       
_________________________________________________________________
gat_conv_1 (GATConv)         multiple                  18        
Total params: 327
Trainable params: 327
Non-trainable params: 0
_________________________________________________________________
Best Val Acc: 0.8042704463005066


NameError: name 'loss_test' is not defined